# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [43]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from train import clean_data
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import pandas as pd
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import json
import requests
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [20]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone_automl'

exp = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136873
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-136873


In [21]:
aml_compute_target = "cpu-cluster"
try:
  aml_compute = AmlCompute(ws, aml_compute_target)
  print("Found existing compute target!")
except ComputeTargetException:
  print("Creating new compute cluster...")
  provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", min_nodes = 1, max_nodes = 6)
  aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
  aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print("Azure Machine Learning Compute Cluster Created!")

Found existing compute target!
Azure Machine Learning Compute Cluster Created!


In [22]:
df = pd.read_csv('https://raw.githubusercontent.com/Anupriya-S/Capstone-Azure-Machine-Learning-Engineer/main/nasa.csv')

In [23]:
df.head()

,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),...,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,3703080,3703080,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,J2000,True
1,3723955,3723955,21.3,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,...,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,J2000,False
2,2446862,2446862,20.3,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,...,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,J2000,True
3,3092506,3092506,27.4,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,...,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,J2000,False
4,3514799,3514799,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,J2000,True


In [24]:
# Use clean_data() for processing the dataframe
df = clean_data(df)

In [27]:
found = False
key = "NASA Asteroids"
description_text = "The data is about Asteroids and is provided by NEOWS(Near-Earth Object Web Service)."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        data_store = Datastore.get_default(ws)
        dataset = TabularDatasetFactory.register_pandas_dataframe(df, target=data_store, name=key, description=description_text, tags=None, show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/ebedc8b3-b4d1-4a59-83b0-daf735d9dc64/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [30]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Hazardous",  
                             n_cross_validations=5,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [31]:
# TODO: Submit your experiment
automl_run = exp.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_2a31a2f6-debc-4d8c-bb34-1b54a7b6c72f



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()

In [38]:
automl_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|755                              |1                                |4687                                  |
+---------------------------------+----------

{'runId': 'AutoML_2a31a2f6-debc-4d8c-bb34-1b54a7b6c72f',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-01T22:21:53.10624Z',
 'endTimeUtc': '2021-02-01T22:44:42.586705Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone_automl","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-136873","workspace_name":"quick-starts-ws-136873","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [44]:
#TODO: Save the best model

best_automl_run, model = automl_run.get_output()
joblib.dump(model,'automl-model.pkl')

['automl-model.pkl']

In [45]:
# details associated with the best AutoML run
print("Run ID:", best_automl_run.id)

# list the model files uploaded during the run
print("\n", best_automl_run.get_file_names())

Run ID: AutoML_2a31a2f6-debc-4d8c-bb34-1b54a7b6c72f_0

 ['accuracy_table', 'automl_driver.py', 'azureml-logs/55_azureml-execution-tvmps_ae3e0d3aa4b5a6aa6fa0970abcd6ce1a5b98bd5497219e0da288e38b3279d9da_d.txt', 'azureml-logs/65_job_prep-tvmps_ae3e0d3aa4b5a6aa6fa0970abcd6ce1a5b98bd5497219e0da288e38b3279d9da_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_ae3e0d3aa4b5a6aa6fa0970abcd6ce1a5b98bd5497219e0da288e38b3279d9da_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'confusion_matrix', 'logs/azureml/105_azureml.log', 'logs/azureml/azureml_automl.log', 'logs/azureml/dataprep/python_span_81dfac35-7778-4bb4-86fa-af1ec1dc1e19.jsonl', 'logs/azureml/dataprep/python_span_c909ca80-8d10-404d-9766-8d9819002680.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/conda_env_v_1_0_0.yml', 'outputs/env_dependencies.json', 'outputs/internal_cross_validated_models.pkl', 'outputs/model.pkl', 'outputs/pipeline_g

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [46]:
registered_model = Model.register(workspace=ws, model_path="automl-model.pkl", model_name="automl-model")

Registering model automl-model


In [50]:
inference_config = InferenceConfig(entry_script='score.py', environment=best_automl_run.get_environment())

In [52]:
service_name = 'asteroid-classification'

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, deployment_target=None, overwrite=True)
service.wait_for_deployment(show_output=True)

print(service.state)
print(service.scoring_uri)

AttributeError: 'Pipeline' object has no attribute 'id'

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = {"data":
        [
          {
            "age": 17,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 971,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 1
          },
          {
            "age": 87,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 471,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 1
          },
      ]
    }

In [ ]:
scoring_uri=service.scoring_uri

input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
response = requests.post(scoring_uri, input_data, headers=headers)
print(response.json())

In [ ]:
prediction = service.run(input_data)

print(prediction)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

In [ ]:
service.delete()

In [ ]:
aml_compute.delete()